<h3>Imports</h3>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

<h3> Load the Pre-trained VGG16 Model and Add Custom Classification Layers</h3>

In [2]:
# Number of classes for the food recognition model
num_of_foods = 12

# Load the VGG16 model pre-trained on ImageNet, without the top layer
base_model = VGG16(
    weights='imagenet',       # Use weights pre-trained on ImageNet
    include_top=False,        # Do not include the top (fully connected) layers
    input_shape=(224, 224, 3) # Input shape must match VGG16's requirements
)

# Freeze the base model to prevent its weights from being updated during training
base_model.trainable = False
# Functional API approach to build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)       # Pass inputs through the VGG16 model

# Add global average pooling to reduce dimensions
x = layers.GlobalAveragePooling2D()(x)

# Add a dense layer for further learning
x = layers.Dense(128, activation='relu')(x)

# Output layer with softmax activation for multi-class classification
outputs = layers.Dense(num_of_foods, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)


<h3>Compile the Model</h3>

In [4]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
    metrics=['accuracy']
)

# Display the model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,781,900 (56.39 MB)

 Trainable params: 67,212 (262.55 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

<h3>Data Generation and Pre-Processing</h3>
<h4>Use ImageDataGenerator to load and preprocess the images.</h4>

In [5]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Rescale pixel values from [0, 255] to [0, 1]
    rotation_range=20,     # Data augmentation options
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # Use 20% of the data for validation
)

# Path to your training dataset directory
data_dir = 'D:/food_dataset'   # Replace with the actual path

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to match VGG16 input size
    batch_size=18,
    class_mode='sparse',     # Use 'sparse' if labels are integers
    subset='training'
)

# Create validation data generator
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 5038 images belonging to 12 classes.
Found 1254 images belonging to 12 classes.


<h3>Train the model</h3>

In [6]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # You can change the number of epochs based on your requirements
    validation_data=validation_generator
)

Epoch 1/10


D:\apps\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


280/280 ━━━━━━━━━━━━━━━━━━━━ 1958s 7s/step - accuracy: 0.3795 - loss: 1.9758 - val_accuracy: 0.6619 - val_loss: 1.2579
Epoch 2/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3124s 11s/step - accuracy: 0.6569 - loss: 1.1490 - val_accuracy: 0.7344 - val_loss: 0.9482
Epoch 3/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3185s 11s/step - accuracy: 0.7357 - loss: 0.9087 - val_accuracy: 0.6595 - val_loss: 0.9421
Epoch 4/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3178s 11s/step - accuracy: 0.7575 - loss: 0.8003 - val_accuracy: 0.7600 - val_loss: 0.7841
Epoch 5/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3157s 11s/step - accuracy: 0.7802 - loss: 0.7085 - val_accuracy: 0.7584 - val_loss: 0.7406
Epoch 6/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3272s 11s/step - accuracy: 0.7909 - loss: 0.6523 - val_accuracy: 0.7616 - val_loss: 0.6930
Epoch 7/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3558s 12s/step - accuracy: 0.8088 - loss: 0.6119 - val_accuracy: 0.7982 - val_loss: 0.6241
Epoch 8/10
280/280 ━━━━━━━━━━━━━━━━━━━━ 3064s 11s/step - accuracy: 0.8062 - loss: 0.5848 - val_

<h3>Evaluate the model</h3>

In [9]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

40/40 ━━━━━━━━━━━━━━━━━━━━ 457s 11s/step - accuracy: 0.8218 - loss: 0.5280
Validation Accuracy: 81.26%
